In [1]:

import pandas as pd
import copy

把XILINX的pkg.txt文件转换为适合orcad 库输入的文件。
使用前需要把pkg.txt的最后一行去掉，以及把列名字的行中的空格去掉
然后在orcad中选择 New Part Creation Spreadsheet.使用生成的exel表格复制粘贴到orcad中。
section number要选文件末尾提示的。


In [2]:
FILE="xc7a100tfgg484pkg.txt"

In [3]:
df=pd.read_csv(FILE,delim_whitespace=True,header=1,dtype=str)
df.sort_values("Bank",inplace=True)
df.fillna("",inplace=True)

In [4]:

abcdstr="ABCDEFGHIJKLMNOPQRSTUVWXUZ"
bank_list = list(set( i for i in df['Bank']))
bank_list.sort()
section_map={ bank_list[i]:abcdstr[i]  for i in range(len(bank_list))}

print(section_map)


{'': 'A', '0': 'B', '13': 'C', '14': 'D', '15': 'E', '16': 'F', '216': 'G', '34': 'H', '35': 'I'}


In [5]:
name_list = list(set( i for i in df['PinName']))
name_count={v:0 for v in name_list}
name_count_zero={v:0 for v in name_list}
for n in df['PinName']:
    name_count[n]=name_count[n]+1


In [6]:
df2=pd.DataFrame()
df2.insert(df2.shape[1],'Number',0)
df2.insert(df2.shape[1],'Name','')
df2.insert(df2.shape[1],'Type','')
df2.insert(df2.shape[1],'Pin Visibility',True)
df2.insert(df2.shape[1],'Shape','')
df2.insert(df2.shape[1],'PinGroup','')
df2.insert(df2.shape[1],'Position','')
df2.insert(df2.shape[1],'Section','')



In [7]:

for i in range(len(df)):
    df2.loc[i,'Number']=df.iloc[i]['Pin']

    name= df.iloc[i]['PinName']
    
    if name_count[name]==1:
        df2.loc[i,'Name']=name
    else:
        df2.loc[i,'Name']=name+"_"+str(name_count_zero[name])
        name_count_zero[name]=name_count_zero[name]+1

    pinname=df.iloc[i]['PinName']

    if "VCC" in pinname or "VTT" in pinname:
        df2.loc[i,'Type']="Power"
        df2.loc[i,'Shape']="Line"
        df2.loc[i,'Position']="Top"
    elif pinname.startswith("GND"):
        df2.loc[i,'Type']="Power"
        df2.loc[i,'Shape']="Line"
        df2.loc[i,'Position']="Bottom"
    else:
        df2.loc[i,'Type']="Bidirectional"
        df2.loc[i,'Shape']="Line"
        df2.loc[i,'Position']="Right"
    df2.loc[i,'Section']=section_map[df.iloc[i]['Bank']]
    df2.loc[i,'Pin Visibility']="1"
    if "CLK" in pinname or "MRCC" in pinname or "DQS" in pinname or "SRCC" in pinname: 
        df2.loc[i,'Shape']="Clock"

In [8]:
df2.sort_values("Name",inplace=True)
df2.to_excel("pin.xlsx")

In [9]:
print(name_count_zero["GND"])
print(name_count["GND"])

87
87


In [10]:
print('total Section is ' ,len(section_map))

total Section is  9
